# 内容
```meeting_0107```のコードを継承

# したいこと
使いたいデータの特徴量を```16```へと減らし、全解探索をする。(※```X```の値によって代わるので、あくまでも今回の```x```の```dataframe```において)

# 方法
特徴量を```1-15```つ選んだときのエネルギーが```0```となる```y_list```の個数の変化を調べる。

# 出力
選ぶ特徴量数ごと、選んだときのエネルギーが```0```となる```y```

# 結果
```python
'training_markers16.txt', 'training_yields16.txt'
```
（特徴量```16```）
横軸：特徴量数、単位：通り
<img src = '20200121210050.png'>

最大13748通り(特徴量を15個選ぶ時)、最少22通り(特徴量を7個選ぶ時)となることがわかった

<br/>

# 追加結果
```python
'training_markers.txt', 'training_yields.txt'
```
（特徴量```20```）のデータにも適用してみた

横軸：特徴量数、単位：通り
<img src = '20200126173200.png'>

最大76422通り(特徴量を15個選ぶ時)、最少12通り(特徴量を10個選ぶ時)となることがわかった

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import math
import itertools

In [2]:
'''
# 説明変数（こっちの特徴量上位〇つ取り出す）
X = pd.read_table('training_markers16.txt', header=None,  sep ='\t')


data_yields = pd.read_table('training_yields16.txt', header=None,  sep ='\t')
#print(data_yields)

# 目的変数
y= data_yields.T.iloc[0]
'''

X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）


data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
#print(data_yields)
ori_y = data_yields.T.iloc[0]
print(ori_y)

y= data_yields.T.iloc[0]

#num_sample = X.shape[0]#サンプル数
#num_feature = X.shape[1]#特徴量の数

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64


In [3]:
print('len(y) = {}'.format(len(y)))
print('type(y) = {}'.format(type(y)))

len(y) = 20
type(y) = <class 'pandas.core.series.Series'>


In [4]:
#特徴量の分散や同じ特徴量が存在するのかを確認

sel = VarianceThreshold() #threshold=0がデフォルト
sel.fit_transform(X) #もし👇で減っていたら面倒なのでsel.fit_transformのほうが良いだろう。

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(sel.get_support())==X.shape[1]) #同じ特徴量は存在していないことがわかった

True


In [5]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    selector = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #f_regression...線形回帰
    selector.fit(X, y)
    mask_each = selector.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask_each

In [6]:
def each_energy(mask_each, num_select): #毎回のmaskとyが引数、合致度合いの計算を行う
    standard_mask = selection(num_select, X, y)
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==standard_mask[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [7]:
selected_X_energy = X.shape[1]#maskのlenなので

In [8]:
print('yの全部の組み合わせは{}通り'.format(2**20))
#だけど全部１と全部０を後で除かないといけない気がする

yの全部の組み合わせは1048576通り


In [9]:
r = range(2) #0, 1なので
y_all_combination = list(itertools.product(r, repeat=20))
y_all_combination[0] #tupleです

len(y_all_combination)==2**20 #True

True

In [10]:
def find_E0(num_select): 
    E0_count = 0
    for y_new in y_all_combination:
        if y_new != (0,) * len(y):
            if y_new != (1,) * len(y):
                y_new = pd.Series(list(y_new))
                mask_new = selection(num_select, X, y_new)
                E_new = selected_X_energy - each_energy(mask_new, num_select)
                if E_new == 0:
                    E0_count += 1
    return E0_count

In [11]:
find_E0_df = pd.DataFrame()
find_E0_df.index.name = 'the number of y which energy is 0'
find_E0_df.columns.name = 'the number of selected features'

find_E0_dic = {}
for num_select in range(1, X.shape[1]):
    E0_count = find_E0(num_select)
    find_E0_dic[num_select] = E0_count
    print('here')

find_E0_df = find_E0_df.append(find_E0_dic, ignore_index=True)

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [12]:
find_E0_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,68666.0,12970.0,2870.0,908.0,214.0,78.0,56.0,18.0,18.0,12.0,34.0,100.0,178.0,568.0,2010.0,2912.0,7464.0,21652.0,76422.0


# 日記
<br/>

-2020/01/27
## 内容
特徴量20のデータに対しても全解探索をした

<br/>

-2020/01/22
## 課題
また```/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide```が発生した

## 解決方法
回帰でなく分類にした


